# BERT Text Classification Model

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
import os

In [1]:
import tensorflow as tf
import os

# Note that the `tpu` argument is for Colab-only
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])

tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

All devices:  [LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:0', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:1', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:2', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:3', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:4', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:5', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:6', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:7', device_type='TPU')]


In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 4.8 MB/s 
     |████████████████████████████████| 7.6 MB 19.0 MB/s 
     |████████████████████████████████| 182 kB 20.8 MB/s 


## Import Library

In [2]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

#Loading the Dataset
#Pre-processing the raw data
#Getting BERT Pre-trained model and its tokenizer
#Training and evaluation
#Prediction Pipeline

In [6]:
df = pd.read_excel("/content/Core Test Data.xlsx")

In [7]:
df.head()

,DocumentId,EventInstanceId,#,News Date,Source,Country,Location (LatLng),Locations,Headline,Abstract,...,Start Date,End Date,Actor List,Location Info,Level of Event,Type,Topic,Aim,Unnamed: 25,Unnamed: 26
0,AgreementsDocId1,AgreementsEventInstanceId1,1.0,2020-11-30 00:00:00,The Libya Observer,libya,NaN,"['Italy', 'Libya', 'Tripoli']",Al-Sarraj reviews with Eni officials support f...,The Head of the Libyan Presidential Council Fa...,...,2020-11-30 00:00:00,NaN,"Al-Sarraj, Eni officials","Tripoli, Libya",Inter Governmental,Meeting,possible investments of the Italian company in...,NaN,NaN,NaN
1,AgreementsDocId2,AgreementsEventInstanceId2,2.0,2020-12-01 00:00:00,The Libya Observer,libya,"(32.806069013310236, 13.201975860087888)","['Italy', 'Libya']","NOC, Eni review resuming stalled projects",Chairman of the National Oil Corporation (NOC)...,...,2020-12-01 00:00:00,NaN,"National Oil Corporation (NOC), Eni company","Tripoli, Libya",Inter Governmental,Meeting,the resumption of some significant projects th...,NaN,NaN,NaN
2,AgreementsDocId3,AgreementsEventInstanceId3,3.0,2020-11-19 00:00:00,Libya Herald,libya,NaN,"['Libya', 'Tripoli', 'Sirte']",Halliburton discusses increased cooperation wi...,"Halliburton Regional Director, Colby Fuser, me...",...,2020-11-18 00:00:00,NaN,"National Oil Corporation (NOC), Colby Fuser","Tripoli, Libya",Inter Governmental,Meeting,development of joint cooperation between the t...,NaN,NaN,NaN
3,AgreementsDocId4,AgreementsEventInstanceId4,4.0,2020-11-19 00:00:00,Bloomberg Business,libya,NaN,"['Libya', 'Sweden', 'France', 'Brega']",Libya Says Total Mulls More Investment in Nati...,Total discussed helping boost flows to highest...,...,2020-11-18 00:00:00,NaN,"National Oil Corporation (NOC), Total, France",Virtual,Inter Governmental,Meeting,efforts to increase Libyan production capacity...,NaN,NaN,NaN
4,AgreementsDocId5,AgreementsEventInstanceId5,5.0,2020-11-18 00:00:00,Libya Herald,libya,NaN,"[""Ra's Lanuf"", 'Tripoli', 'Benghazi', 'Libya']",Zallaf and NOC discuss new South Refinery,A wide ranging face-to-face technical meeting ...,...,2020-11-17 00:00:00,NaN,"National Oil Corporation (NOC), Zallaf Libya Oil","Tripoli, Libya",Intra Governmental,Meeting,the technical and financial details of the pro...,NaN,NaN,NaN


In [8]:
df2 = df[df["Event Category"] == "politicalEvents"][["Headline","Abstract","First Part","Body","Event Category","1st Level Sub Category","2nd Level Sub Category","3rd Level Sub Category"]]

In [9]:
df2["Input"]=df2["Headline"].map(str) +" "+ df2["Abstract"].map(str) +" "+ df2["First Part"].map(str) +" " + df2["Body"].map(str)

In [10]:
df2["Input"].fillna('', inplace=True)

In [11]:
df2["1st Level Sub Category"].value_counts(dropna=False)

election            13
governmentChange     8
badGovernance        7
goodGovernance       5
Name: 1st Level Sub Category, dtype: int64

In [33]:
df2=df2.sample(frac=1)

In [34]:
df2["1st Level Sub Category"]

375       badGovernance
386    governmentChange
380      goodGovernance
391    governmentChange
379      goodGovernance
384            election
372       badGovernance
388    governmentChange
367            election
392    governmentChange
362            election
373       badGovernance
361            election
371       badGovernance
389    governmentChange
383            election
378            election
370       badGovernance
366            election
382      goodGovernance
387    governmentChange
364            election
377      goodGovernance
374       badGovernance
360            election
363            election
381      goodGovernance
365            election
385    governmentChange
368            election
390    governmentChange
376            election
369       badGovernance
Name: 1st Level Sub Category, dtype: object

In [35]:
df2_train=df2.iloc[0:22]

In [36]:
df2_test=df2[22:34]

#Converting our 1st Level Sub Category column into Categorical data



In [37]:
encoded_dict = {'election':0,'badGovernance':1, 'goodGovernance':2, 'governmentChange':3}
df2_train['1st Level Sub Category']= df2_train["1st Level Sub Category"].map(encoded_dict)
df2_test['1st Level Sub Category'] = df2_test['1st Level Sub Category'].map(encoded_dict)

In [38]:
df2_test

,Headline,Abstract,First Part,Body,Event Category,1st Level Sub Category,2nd Level Sub Category,3rd Level Sub Category,Input
377,Supporting Democratic Transition through Fair ...,Inclusive elections are one of the most basic ...,"HNEC, in partnership with the United Nations S...",NaN,politicalEvents,2,NaN,NaN,Supporting Democratic Transition through Fair ...
374,Human Rights Watch: Libya 'incapable' of holdi...,Libya is not ready to hold free and fair elect...,The country's rival authorities are planning t...,NaN,politicalEvents,1,NaN,NaN,Human Rights Watch: Libya 'incapable' of holdi...
360,Former TV repairman Naker announces his candid...,"The Leader of the 'Summit Party', Abdullah Nak...",Regarding methods of communication with his Su...,NaN,politicalEvents,0,NaN,NaN,Former TV repairman Naker announces his candid...
363,Libyans to hold national elections in December...,bodies and groups from across Libya's politica...,"""It's a very important date for Libyans,"" she ...",The announcement marked the first concrete dec...,politicalEvents,0,NaN,NaN,Libyans to hold national elections in December...
381,Post-war reconstruction plans for Libya?,An assessment trip is planned within two weeks...,"""Separately, a meeting between Libyan business...",NaN,politicalEvents,2,NaN,NaN,Post-war reconstruction plans for Libya? An as...
365,Libyans agree preliminary plan for elections: ...,Libyans agreed in UN-led talks on Wednesday a ...,The talks in Tunisia aim to create a framework...,The UN selected the 75 invitees to the politic...,politicalEvents,0,NaN,NaN,Libyans agree preliminary plan for elections: ...
385,UN-led talks elect Libyan transitional governm...,The EU hailed a ‘milestone’ as Libya embarked ...,In a potential turning point accord widely wel...,NaN,politicalEvents,3,NaN,NaN,UN-led talks elect Libyan transitional governm...
368,Moin Kikhia Announces Candidacy for Libya’s Un...,"Moin Kikhia, the Founder of the Libyan Democra...",“I have submitted my candidacy for the role of...,NaN,politicalEvents,0,NaN,NaN,Moin Kikhia Announces Candidacy for Libya’s Un...
390,Libya Security Chief Suspended in Escalating P...,Libya’s presidential council suspended its int...,The internationally-recognized government said...,NaN,politicalEvents,3,NaN,NaN,Libya Security Chief Suspended in Escalating P...
376,"Mayors call for elections by February 2021, ex...",Twenty-nine elected mayors from cities and reg...,They also accused the current political elite ...,NaN,politicalEvents,0,NaN,NaN,"Mayors call for elections by February 2021, ex..."


In [39]:
from tensorflow.keras.utils import to_categorical

In [40]:
y_train = to_categorical(df2_train["1st Level Sub Category"])
y_test = to_categorical(df2_test["1st Level Sub Category"])

#Loading Model and Tokenizer from the transformers package 

In [41]:
from transformers import AutoTokenizer,TFBertModel
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
bert = TFBertModel.from_pretrained("bert-base-cased")

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


#Input Data Modeling
Before training, we need to convert the input textual data into BERT’s input data format using a tokenizer.
Tokenizer takes all the necessary parameters and returns tensor in the same format Bert accepts.

return_token_type_ids = False: token_type_ids is not necessary for our training in this case.

return_attention_mask = True we want to include attention_mask in our input.

return_tensors=’tf’: we want our input tensor for the TensorFlow model.

max_length=512:

add_special_tokens=True, CLS, SEP token will be added in the tokenization.

the tokenizer will return a dictionary (x_train) containing ‘Input_ids’, ‘attention_mask’ as key for their respective
data.

In [42]:
# Tokenize the input (takes some time) 
# here tokenizer using from bert-base-cased
x_train = tokenizer(
    text=df2_train.Input.tolist(),
    add_special_tokens=True,
    max_length=512,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)
x_test = tokenizer(
    text=df2_test.Input.tolist(),
    add_special_tokens=True,
    max_length=512,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

In [43]:
input_ids = x_train["input_ids"]
attention_mask = x_train['attention_mask']

#Model Building
Bert layers accept three input arrays, input_ids, attention_mask, token_type_ids

input_ids means our input words encoding, then attention mask,

token_type_ids is necessary for the question-answering model; in this case, we will not pass token_type_ids.

For the Bert layer, we need two input layers, in this case, input_ids, attention_mask.

Embeddings contain hidden states of the Bert layer.
using

GlobalMaxPooling1D then dense layer to build CNN layers using hidden states of Bert. 
These CNN layers will yield our output.

bert[0] is the last hidden state, bert[1] is the
pooler_output, for building CNN layers on top of the BERT layer, Bert’s hidden forms are used.

In [44]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense

In [45]:
max_len = 512
input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
input_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")
embeddings = bert(input_ids,attention_mask = input_mask)[0] 
out = tf.keras.layers.GlobalMaxPool1D()(embeddings)
out = Dense(128, activation='relu')(out)
out = tf.keras.layers.Dropout(0.1)(out)
out = Dense(32,activation = 'relu')(out)
y = Dense(4,activation = 'softmax')(out)
model = tf.keras.Model(inputs=[input_ids, input_mask], outputs=y)
model.layers[2].trainable = True

#Defining learning parameters and compiling the model.
learning_rate = 5e-05 the learning rate for the model will be significantly lower.

Loss = CategoricalCrossentropy since we are passing the categorical data as the target.

Balanced accuracy will take care of our average accuracy for all the classes.

In [46]:
optimizer = Adam(
    learning_rate=5e-05, # this learning rate is for bert model , taken from huggingface website 
    epsilon=1e-08,
    decay=0.01,
    clipnorm=1.0)
# Set loss and metrics
loss =CategoricalCrossentropy(from_logits = True)
metric = CategoricalAccuracy('balanced_accuracy'),
# Compile the model
model.compile(
    optimizer = optimizer,
    loss = loss, 
    metrics = metric)

In [47]:
y_train.shape

(22, 4)

#Model Training
with x_train, y_train we can now train the model.
model.fit returns a history object which keeps all the training history.

x_test became a dictionary containing ‘input_ids’, ‘attention_mask‘ after pre-processing. 
We are passing input_ids and attention_mask for the training.

In the validation data, we are passing the test data.

In [66]:
train_history = model.fit(
    x ={'input_ids':x_train['input_ids'],'attention_mask':x_train['attention_mask']} ,
    y = y_train,
    validation_data = (
    {'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']}, y_test
    ),
  epochs=1,
    batch_size=2
)

11/11 [==============================] - 169s 15s/step - loss: 0.2740 - balanced_accuracy: 0.9091 - val_loss: 1.2571 - val_balanced_accuracy: 0.4545


#Model Evaluation

In [67]:
predicted_raw = model.predict({'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']})
predicted_raw[0]

1/1 [==============================] - 21s 21s/step


array([0.24573366, 0.5649744 , 0.06565352, 0.1236384 ], dtype=float32)

In [68]:
y_predicted = np.argmax(predicted_raw, axis = 1)
y_true = df2_test["1st Level Sub Category"]

#Classification Report

In [69]:
from sklearn.metrics import classification_report
print(classification_report(y_true, y_predicted))

              precision    recall  f1-score   support

           0       1.00      0.60      0.75         5
           1       0.00      0.00      0.00         2
           2       0.00      0.00      0.00         2
           3       0.29      1.00      0.44         2

    accuracy                           0.45        11
   macro avg       0.32      0.40      0.30        11
weighted avg       0.51      0.45      0.42        11



# Prediction Pipeline
Converting indexes back to the labels:


In [91]:
texts = input(str('input the text'))
x_val = tokenizer(
    text=texts,
    add_special_tokens=True,
    max_length=512,
    truncation=True,
    padding='max_length', 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True) 
validation = model.predict({'input_ids':x_val['input_ids'],'attention_mask':x_val['attention_mask']})*100
for key , value in zip(encoded_dict.keys(),validation[0]):
    print(key,value)

input the textThis joint project is a testament to the shared commitment of the European Union, UNODC and UNICRI to support Libya in building long-term sustainable national capacities to counter corruption and money laundering. This new intervention supplements and expands UNODC\'s long-standing partnership with Libyan Authorities in countering trafficking in persons and smuggling of migrants, related financial crimes and terrorism financing as well as in supporting prison reform." stated Ms. Cristina Albertin, UNODC Regional Representative for the Middle East and North Africa, while welcoming the project. Ms. Antonia Marie De Meo, UNICRI\'s Director, highlighted that "Libyans have suffered from corruption for too long. They deserve accountability and transparency from all public officials. The Libyan people should also benefit from the recovery of stolen assets which - when recovered - can be directed to key public sectors such as health care and education." UNICRI has already assiste

#Actual class is `governmentChange`
#Predicted class is `governmentChange`

In [90]:
df[df["Event Category"] == "politicalEvents"][["First Part","Body","Event Category","1st Level Sub Category"]].tail(11)

,First Part,Body,Event Category,1st Level Sub Category
382,Authorities will also be trained in techniques...,"""This joint project is a testament to the shar...",politicalEvents,goodGovernance
383,When the Supreme Election Commission initially...,NaN,politicalEvents,election
384,Speaking to reporters during a press conferenc...,NaN,politicalEvents,election
385,In a potential turning point accord widely wel...,NaN,politicalEvents,governmentChange
386,UN sponsored talks produced a new interim gove...,NaN,politicalEvents,governmentChange
387,Ever since the collapse of the old regime and ...,NaN,politicalEvents,governmentChange
388,According to the Prime Minister’s press office...,NaN,politicalEvents,governmentChange
389,The struggle for power within the Libya Presid...,NaN,politicalEvents,governmentChange
390,The internationally-recognized government said...,NaN,politicalEvents,governmentChange
391,He did not spell out in his brief televised sp...,Haftar is supported by the United Arab Emirate...,politicalEvents,governmentChange


In [88]:
df[df["Event Category"] == "politicalEvents"][["First Part","Body","Event Category","1st Level Sub Category"]]["Body"].loc[382]

'"This joint project is a testament to the shared commitment of the European Union, UNODC and UNICRI to support Libya in building long-term sustainable national capacities to counter corruption and money laundering. This new intervention supplements and expands UNODC\'s long-standing partnership with Libyan Authorities in countering trafficking in persons and smuggling of migrants, related financial crimes and terrorism financing as well as in supporting prison reform." stated Ms. Cristina Albertin, UNODC Regional Representative for the Middle East and North Africa, while welcoming the project. Ms. Antonia Marie De Meo, UNICRI\'s Director, highlighted that "Libyans have suffered from corruption for too long. They deserve accountability and transparency from all public officials. The Libyan people should also benefit from the recovery of stolen assets which - when recovered - can be directed to key public sectors such as health care and education." UNICRI has already assisted the Govern